<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/iCarl(risultati%20slicing%20dal%20secondo%20batch%2C%20no%20classificatore).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install 'import_ipynb'
!pip3 install 'tqdm'

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import math
from sklearn.preprocessing import normalize
import copy
import torchvision.datasets as dsets
import torchvision.models as models
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from torch.utils.data import Subset, DataLoader
import random
from sklearn.metrics import confusion_matrix as s_cm
import seaborn as sn
import pandas as pd


  

import import_ipynb
from cifar100 import ilCIFAR100

from resnet32 import resnet32
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

importing Jupyter notebook from cifar100.ipynb
Files already downloaded and verified
importing Jupyter notebook from resnet32.ipynb


In [26]:
class icarl(nn.Module):
  def __init__(self,n_classes=100):
    super(icarl, self).__init__()
    self.model = resnet32(num_classes=n_classes)
    self.feature_extractor = self.model.features
    self.lr = 2
    self.gamma = 0.2
    self.weight_decay =1e-5 
    self.milestones = [49,63]
    self.batch_size = 128
    self.numepochs = 70
    self.n_classes = 0
    self.n_known = 0
    self.feature_size=64
    self.momentum=0.9
    self.criterion=nn.BCEWithLogitsLoss()
    self.compute_means = True
    self.exemplar_means = []
    self.exemplar_sets = []
    self.NUM_BATCHES=10
    self.randomseed=981
    self.trainloader=None
    self.testloader=None
    self.CLASSES_PER_BATCH=10
    self.original_training_set = ilCIFAR100(self.CLASSES_PER_BATCH,self.randomseed)
    self.original_test_set = ilCIFAR100(self.CLASSES_PER_BATCH,self.randomseed, train=False)
    
    self.classes_seen=0
    self.diz = self.original_training_set.get_dict()

  def update_parameters(self):
    old_model = copy.deepcopy(self)
    old_model.eval()
    old_model.to('cuda')
    n_classes=self.classes_seen+self.CLASSES_PER_BATCH
    print(n_classes)
    optimizer = optim.SGD(self.model.parameters(), lr=self.lr, momentum=self.momentum, weight_decay=self.weight_decay)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.milestones, gamma=self.gamma)
    for epoch in tqdm(range(self.numepochs)):
      #print(epoch)
        
      for _,inputs,labels in self.trainloader:
        inputs = Variable(torch.FloatTensor(inputs)).cuda()
        labels = torch.tensor([torch.tensor(self.diz[c.item()]) for c in labels])
        #print(labels)
        labels=Variable(labels).to('cuda')
        optimizer.zero_grad()
        outputs=self.forward(inputs)

        labels_encoded = F.one_hot(labels,100).float().cuda() #CAMBIARE ONE_HOT
        
        if self.classes_seen:
          old_target = old_model.forward(inputs).cuda()
          old_target = torch.sigmoid(old_target).cuda()
          
          target = torch.cat((old_target[:,:self.classes_seen], labels_encoded[:, self.classes_seen:n_classes]), dim=1)
          loss = self.criterion(outputs[:,:n_classes], target)
        else:
          loss=self.criterion(outputs,labels_encoded) 

        loss.backward()
        optimizer.step()
      
      scheduler.step()
  def get_new_exemplars(self,batch,m):
    loader=torch.utils.data.DataLoader(batch, batch_size=self.batch_size,shuffle=False, num_workers=4)
    features = np.zeros((0,self.feature_size))
    indices = np.zeros((0), dtype=int)
    with torch.no_grad():
      for indexes,images,labels in loader:
        images = images.cuda()
        feature = self.feature_extractor(images).data.cpu().numpy()
        feature = normalize(feature, axis=1, norm='l2')
        features = np.concatenate((features,feature), axis=0)
        indices = np.concatenate((indices,indexes), axis=0)

    class_mean = np.mean(features, axis=0)
    class_mean = class_mean / np.linalg.norm(class_mean)  # Normalize

    exemplar_set = []
    exemplar_features = np.zeros((0,self.feature_size))

    for k in range(1, int(m)+1):
        S = np.sum(exemplar_features, axis=0)
        phi = features
        mu = class_mean
        mu_p = 1.0 / k * (phi + S)
        mu_p = normalize(mu_p, axis=1, norm='l2')
        i = np.argmin(np.sqrt(np.sum((mu - mu_p) ** 2, axis=1)))
        exemplar_set.append(indices[i])
        addfeature =  np.expand_dims(features[i], axis=0)
        exemplar_features = np.concatenate((exemplar_features,addfeature), axis=0)

        #remove duplicates
        features = np.delete(features, i, 0)
        indices = np.delete(indices, i, 0)
        
    self.exemplar_sets.append(exemplar_set)
        


  def reduce_old_exemplars(self,m):
    for y, P_y in enumerate(self.exemplar_sets):
            self.exemplar_sets[y] = P_y[:int(m)]

  def forward(self, x):
    self.model = self.model.cuda()
    return self.model.forward(x)
  def classify(self, image):
    _, preds = torch.max(torch.softmax(self.forward(image), dim=1), dim=1, keepdim=False)
    return preds
  def __accuracy_on(self, dl, model, mapper):
    total = 0.0
    correct = 0.0
    for  _,images, labels in dl:
        labels = torch.tensor([torch.tensor(mapper[c.item()]) for c in labels])
        images = Variable(images).cuda()
        preds = model.classify(images)
        total = total + len(labels)
        correct += (preds.data.cpu() == labels).sum()

    acc = 100 * correct / total
    return acc
  def training_model(self):
    
    train_indices = self.original_training_set.get_batch_indexes()
    test_indices = self.original_test_set.get_batch_indexes()
    batches=self.original_training_set.getbatches()
    current_test_indexes=[]
    acc=[]
    accuracy=0
    for i in range(self.NUM_BATCHES):
      
      for label in self.exemplar_sets:
        train_indices[i]=np.concatenate((train_indices[i],label),axis=0)
      print(len(train_indices[i]))
      train_dataset=Subset(self.original_training_set,train_indices[i])
      current_test_indexes+=test_indices[i].tolist()
      test_dataset=Subset(self.original_test_set,current_test_indexes)
      self.trainloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4, drop_last=True)
      self.testloader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4, drop_last=True)        
      self.train()
      self.update_parameters()
      print('si dio icarl')      
      self.classes_seen+=10
      self.eval() # Set Network to evaluation mode
      print('accuracy on training set:', self.__accuracy_on(self.trainloader,self,self.diz))
      print('accuracy on test set:', self.__accuracy_on(self.testloader,self,self.diz))
      print('-' * 80)
      acc.append(accuracy)
      m=int(2000/(int(i*10+10)))
      self.reduce_old_exemplars(m)
      for classlabel in batches[i]:
        indexes_class=self.original_training_set.get_class_indexes(classlabel)
        #print(indexes_class[7])
        current_class=Subset(self.original_training_set,indexes_class)
        self.get_new_exemplars(current_class,m)
    return self

In [27]:
model=icarl().cuda()
model.training_model()

Files already downloaded and verified
Files already downloaded and verified
5000
10



si dio icarl
accuracy on training set: tensor(99.4591)
accuracy on test set: tensor(86.6071)
--------------------------------------------------------------------------------
7000
20



si dio icarl
accuracy on training set: tensor(96.0214)
accuracy on test set: tensor(77.1354)
--------------------------------------------------------------------------------
7000
30



si dio icarl
accuracy on training set: tensor(94.8351)
accuracy on test set: tensor(71.7391)
--------------------------------------------------------------------------------
6980
40



si dio icarl
accuracy on training set: tensor(92.4045)
accuracy on test set: tensor(65.3982)
--------------------------------------------------------------------------------
7000
50


...


si dio icarl
accuracy on training set: tensor(87.2251)
accuracy on test set: tensor(50.2749)
--------------------------------------------------------------------------------
6960
80



si dio icarl
accuracy on training set: tensor(89.4531)
accuracy on test set: tensor(46.1946)
--------------------------------------------------------------------------------
7000
90



si dio icarl
accuracy on training set: tensor(88.3681)
accuracy on test set: tensor(42.0312)
--------------------------------------------------------------------------------
6980
100



si dio icarl
accuracy on training set: tensor(86.4583)
accuracy on test set: tensor(39.6635)
--------------------------------------------------------------------------------


icarl(
  (model): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_siz